# Dedicated Operators

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)

In [36]:
import brainpy as bp
import brainpy.math as bm

bm.set_platform('cpu')

In [45]:
data = (bm.arange(7) + 1).astype(float)
indices = bm.array([2, 3, 4, 0, 2, 4, 1])
indptr = bm.array([0, 3, 6, 7])

In [46]:
event = bm.array([True, True, False])

In [47]:
bm.event.csrmv(data, indices, indptr, event, shape=(3, 5), transpose=True)

DeviceArray([4., 0., 6., 2., 9.], dtype=float32)

In [51]:
bm.pre2post_event_sum(event, (indices, indptr), post_num=5, values=data)

DeviceArray([4., 0., 6., 2., 9.], dtype=float32)

In [54]:
from timeit import default_timer

pre_num = post_num = 100000

data = bm.arange(pre_num)
indices = bm.arange(pre_num)
indptr = bm.arange(pre_num + 1)

event = bm.random.bernoulli(p=0.25, size=pre_num)

start = default_timer()

post = bm.pre2post_event_sum(event, (indices, indptr), post_num=post_num, values=data)

duration = default_timer() - start

print('time spent: {} s'.format(duration))

time spent: 0.17084040000008827 s


In [55]:
start = default_timer()

post = bm.event.csrmv(data, indices, indptr, event, shape=(pre_num, pre_num), transpose=True)

duration = default_timer() - start

print('time spent: {} s'.format(duration))

time spent: 0.0011178999998264771 s


## Fast computation of synaptic transmission with specific connection patterns

In [33]:
pre_num, post_num = 50000, 10000
weight = 1.
pre_event = bm.random.bernoulli(p=0.2, size=pre_num)
conn_prob = 0.1

start = default_timer()

# generate the connection matrix
conn = bp.connect.FixedProb(prob=conn_prob)(pre_num, post_num)
conn_matrix = conn.require('conn_mat')

# compute the product
post = bm.matmul(pre_event, weight * conn_matrix)

duration = default_timer() - start

print('time spent: {} s'.format(duration))

time spent: 4.293853899999931 s


In [34]:
start = default_timer()

# generate the connection with the pre2post structure
conn = bp.connect.FixedProb(prob=conn_prob)(pre_num, post_num)
pre2post = conn.require('pre2post')

# compute the event-driven synaptic summation
post = bm.pre2post_event_sum(pre_event, pre2post=pre2post, post_num=post_num, values=weight)

duration = default_timer() - start

print('time spent: {} s'.format(duration))

time spent: 3.794358599999896 s


In [49]:
start = default_timer()

# the connection is generated during computation, 
# so the entire connection information is not required to be stored or accessed
# post = bm.event_matvec_prob_conn_homo_weight(pre_event, weight=weight, conn_prob=conn_prob, 
#                                              shape=(pre_num, post_num), transpose=True)
bm.jitconn.event_mv_prob_homo(pre_event, weight=weight, conn_prob=conn_prob, 
                              shape=(pre_num, post_num), transpose=True)

duration = default_timer() - start

print('time spent: {} s'.format(duration))

time spent: 0.3703867000001537 s
